# Libraries and Global Variables

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

import arcpy

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [2]:
# web app folder
folderWebAppDataFolder = r"..\widgets\Resiliency\data"

In [3]:
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )

print(data_folder)
print(intermediate_folder)
print(results_folder)

e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\data
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\intermediate
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\results


In [4]:
processGdb = os.path.join(intermediate_folder,'process.gdb')
processGdb

'e:\\WebAppBuilderForArcGIS_v2.20\\server\\apps\\15\\_dataprep\\intermediate\\process.gdb'

In [5]:
def delIfExists(obj):
    try:
        if arcpy.Exists(obj):
            arcpy.Delete_management(obj)
            #print ("DELTED: " + obj)
        #else:
            #print ("DOESN'T EXIST: " + obj)
    except:
        #print ("PASSED: " + obj)
        pass

# Download Data

In [6]:
dfCategories = pd.DataFrame([
  ["AF", "Agricultural and Farmland"            ],
  ["CH", "Cultural and Historic Resources"      ],
  ["EJ", "Environmental Justice Consideration"  ],
  ["FP", "Floodplains"                          ],
  ["GH", "Geological Hazards"                   ],
  ["HW", "Habitat and Wildlife"                 ],
  ["HM", "Hazardous Materials & Contamination"  ],
  ["HY", "Hydrological"                         ],
  ["OS", "Open Space, Parks, and Recreation"    ],
# ["LU", "Land Use"                             ], # DON'T USE FOR NOW
  ["SS", "Steep Slopes"                         ]
], columns=['CategoryCode','CategoryName'])
display(dfCategories)

# GOOGLE SHEEET
#https://docs.google.com/spreadsheets/d/1cpS4wcMXwt9Yd7qZisvJfZ8GcRLhFpcTRJS_l7CBBY0/edit#gid=0

# quick view layers: https://www.arcgis.com/apps/mapviewer/index.html?layers=74a6478eae424c17b96d1677ee1a4654

dfOpenDataLayers = pd.DataFrame([  
  ["OS", "LOWN", "State and Federal Lands"              , "State and Federal Lands"              , "929fe39efa1d43ac8b151c2898ad3efe", "OWNER!='Private'"       , 1.5],
  ["AF", "AGFL", "Agricultural and Farmland"            , "Agricultural and Farmland"            , "875527c1e41e40d8b52340f18e89fb52", ""                       , 1.5],
  ["HW", "ELKS", "Elk"                                  , "Elk Habitat"                          , "224dbcbdca174d3e8648a8b5693fd130", "VALUE=='crucial'"       , 1.5],
  ["HW", "DEER", "Mule Deer"                            , "Mule Deer Habitat"                    , "a4ea235cef824862b560c693b4f7959b", "VALUE=='crucial'"       , 1.5],
 #["HW", "PRNG", "Pronghorn"                            , "Pronghorn Habitat"                    , "1cd6e0315a19473d925427f7cc1d02c1", "VALUE=='crucial'"       , 1.5], # no near WFRC
 #["HW", "TURK", "Turkey"                               , ""                                     , "9654e665cccd4884adb6a416df4d3444", ""                       , 1.5], # no turkeys in crucial
  ["HW", "MOOS", "Moose"                                , "Moose Habitat"                        , "e66c061ebd4346e3b16ed8786f5d5d78", "VALUE=='crucial'"       , 1.5],
  ["HW", "BEAR", "Black Bear"                           , "Black Bear Habitat"                   , "6440f19a05c149b783baf1705e5f2a51", "VALUE=='crucial'"       , 1.5],
 #["HW", "HARE", "Snowshoe Hare"                        , "Snowshoe Hare Habitat"                , "d151c91e95c74b329814319f3c20b145", "VALUE=='crucial'"       , 1.5], # none near WFRC
  ["HW", "GOAT", "Mountain Goat"                        , "Mountain Goat Habitat"                , "b6b369f865d243399bfc80908426d31b", "VALUE=='crucial'"       , 1.5],
  ["HW", "DGRS", "Dusky Grouse"                         , "Dusky Grouse Habitat"                 , "10382c7f5f384fc692d094fcb65bf251", "VALUE=='crucial'"       , 1.5],
 #["HW", "RGRS", "Ruffed Grouse"                        , ""                                     , "a3d3fe1f13324882aa4b7f67253b1c06", ""                       , 1.5], # no crucial near WF
 #["HW", "HPAR", "Hungarian Partridge"                  , ""                                     , "7a7c3a873f0148ab8a188f743b7acff2", ""                       , 1.5], # no crucial
  ["HW", "CHKR", "Chukar"                               , "Chukar Habitat"                       , "97c6f7c74f704f84b687eb0746cc07ec", "VALUE=='crucial'"       , 1.5],
  ["HW", "BISN", "Bison"                                , "Bison Habitat"                        , "9bed043eb87641db88424d1080d03045", "VALUE=='crucial'"       , 1.5],
 #["HW", "BGHR", "Rocky Mountain Bighorn Sheep"         , ""                                     , "82b1e7ce1ec74920bcd40cd7e729c9b3", ""                       , 1.5], # none near WFRC
 #["HW", "RNPH", "Ringed-Neck Pheasant"                 , ""                                     , "56acb32933f4463bb81cebb834595905", ""                       , 1.5], # none near WF
  ["HW", "CSTG", "Columbian Sharp-Tailed Grouse"        , "Columbian Sharp-Tailed Grouse Habitat", "c8e7ae2533d84a9a98268357dafa15da", "VALUE=='crucial'"       , 1.5],
  ["HW", "CAQU", "California Quail"                     , "California Quail Habitat"             , "d5dae6ecfc8d490b87f1f67fd4d16fd9", "VALUE=='crucial'"       , 1.5],
  ["HW", "CABS", "California Bighorn Sheep"             , "California Bighorn Sheep Habitat"     , "4c11e02a83f148dcafacbe203efc7432", ""                       , 1.5], # include all since value field looks mis-coded
  ["HW", "BTPG", "Band-Tailed Pigeon"                   , "Band-Tailed Pigeon Habitat"           , "524a4c4b704244d4979b4d2b56f1d937", "VALUE=='crucial'"       , 1.5],
 #["HW", "GMQU", "Gambel's Quail"                       , ""                                     , "95a538966055498e956817bad0a5835c", ""                       , 1.5], # None near WF
 #["HW", "WTPT", "White-Tailed Ptarmigan"               , ""                                     , "170dbdc20fcc4a138022738167c4a765", ""                       , 1.5], # None near WF
 #["HW", "WWDV", "White-Winged Dove"                    , ""                                     , "a2d03a434d6b4b189784434014fe0eef", ""                       , 1.5], # None near WF
 #["HW", "UDBH", "Utah Desert Bighorn Sheep"            , ""                                     , "8c19124fbce245b5942c9e48c0568fab", ""                       , 1.5], # None near WF
 #["HW", "WTJR", "White-Tailed Jackrabbit"              , ""                                     , "1b02a508fa334d33ba18c0156d027751", ""                       , 1.5], # None near WF
  ["HM", "BFWT", "Brownfields Targeted for Cleanup"     , "Brownfields Targeted for Cleanup"     , "630351c0db444fe7933a08eab40e573e", ""                       , 1.5],
  ["HM", "BFNT", "Brownfields Not Targeted for Cleanup" , "Brownfields Not Targeted for Cleanup" , "5fd65304423d40b38b4d93e50698db70", ""                       , 1.5],
  ["HM", "HZWO", "Hazardous Waste & Used Oil Facilities", "Hazardous Waste & Used Oil Facilities", "8e9e6310622043bca68ffc2298959126", ""                       , 1.5],
 #["HM", "LLWD", "Low-Level Waste & Uranium Mills"      , "Low-Level Waste & Uranium Mills"      , "2c81695cbc644afdba7687e0340b5763", ""                       , 1.5], # DO NOT USE
  ["HM", "SWLN", "Solid Waste Landfills"                , "Solid Waste Landfills"                , "1ce0d598a0e440558c63535661dc2399", ""                       , 1.5],
  ["HM", "FUDS", "Formerly Used Defense Sites"          , "Formerly Used Defense Sites"          , "dbcdc13c51614d109102f1251fe9530b", ""                       , 1.5],
  ["HM", "MMRP", "Military Munitions Response Program"  , "Military Munitions Response Program"  , "c77c53794aeb4c448e1307b31c581909", ""                       , 1.5],
 #["HM", "EWAS", "Enforceable Written Assurances"       , "Enforceable Written Assurances"       , "7299aa0fdc80426ca0e42c42f59e1518", ""                       , 1.5],
  ["HM", "NPRL", "National Priorities List"             , "National Priorities List"             , "f9c01f05050c40d7b86dade7dd877cf9", ""                       , 1.5],
  ["HM", "SFST", "Superfunded Sites"                    , "Superfunded Sites"                    , "929a2e7a8d5347b8b3248a450cc42e33", ""                       , 1.5],
  ["HM", "ICBZ", "Institutional Control Buffer Zones"   , "Institutional Control Buffer Zones"   , "d8ea2ca541724277b01726d98197da32", ""                       , 1.5],
 #["HM", "UTRI", "Utah Toxic Release Inventory Tier II" , "Utah Toxic Release Inventory Tier II" , "2f725ff758e24dc496b17f87b7301ed8", ""                       , 1.5],
  ["HM", "TXRI", "Toxic Release Inventory"              , "Toxic Release Inventory"              , "132ac94ba71d4e9b9bcffa57b07feb54", ""                       , 1.5],
  ["HM", "VCUP", "Voluntary Cleanup Program"            , "Voluntary Cleanup Program"            , "e0316bc7899d4e4896e88fe3a46aafa8", ""                       , 1.5],
 #["HM", "PSTF", "Petroleum Storage Tank Facilities"    , "Petroleum Storage Tank Facilities"    , "4d31fef4543f449ebe04a84909a11023", ""                       , 1.5],
  ["CH", "UHDS", "Utah Historic Districts"              , "Utah Historic Districts"              , "eecae7fdf65f47aeae12ba817aa0d66b", ""                       , 1.5],
  ["EJ", "EQFA", "Equity Focus Areas"                   , "Equity Focus Areas"                   , "2ed6d9b418d942b79323e6be7209d63c", ""                       , 1.5],
  ["OS", "LCPK", "Local Parks"                          , "Local Parks"                          , "818536733987486b8082e4c5dec0a1dd", ""                       , 1.5],
  ["OS", "PAPR", "Public Access Properties (WMA)"       , "Public Access Properties (WMA)"       , "56aadb826ef3442db805553caf8126c0", ""                       , 1.5],
  ["FP", "FLDP", "Floodplains"                          , "Floodplains"                          , "9100130e33034813833a938069b7bf79", ""                       , 1.5],
  ["GH", "QUFL", "Quaternary Faults"                    , "Quaternary Faults"                    , "9c85978d0fb54570bc60bec467e2fa7f", ""                       , 1.5],
  ["SS", "STSL", "Steep Slopes"                         , "Steep Slopes"                         , "d42f60e142dd4366a1c24aae4993f8e3", ""                       , 1.5],
  ["HY", "RIPA", "Riparians"                            , "Riparians"                            , "58dc87c8dcff457d899ce0943750f352", ""                       , 1.5],
  ["HY", "WETL", "Wetlands"                             , "Wetlands"                             , "4fc2862f90064eb898d5b70861ab7174", ""                       , 1.5]
],columns=['CategoryCode','LayerCode','ListName','LayerName','OpenDataKey','QueryFilter','SearchBuffer'])
display(dfOpenDataLayers)

# example: https://opendata.arcgis.com/datasets/a37e501176dd400c9c7498c29e766ba2_0.gdb

#item  = gis.content.get("630351c0db444fe7933a08eab40e573e")
#layer = item.layers[0]

#dfDownloadedLayers = pd.DataFrame([
#  [],
#  [],
#],columns=['LayerName','Filename'])

,CategoryCode,CategoryName
0,AF,Agricultural and Farmland
1,CH,Cultural and Historic Resources
2,EJ,Environmental Justice Consideration
3,FP,Floodplains
4,GH,Geological Hazards
5,HW,Habitat and Wildlife
6,HM,Hazardous Materials & Contamination
7,HY,Hydrological
8,OS,"Open Space, Parks, and Recreation"
9,SS,Steep Slopes


,CategoryCode,LayerCode,ListName,LayerName,OpenDataKey,QueryFilter,SearchBuffer
0,OS,LOWN,State and Federal Lands,State and Federal Lands,929fe39efa1d43ac8b151c2898ad3efe,OWNER!='Private',1.5
1,AF,AGFL,Agricultural and Farmland,Agricultural and Farmland,875527c1e41e40d8b52340f18e89fb52,,1.5
2,HW,ELKS,Elk,Elk Habitat,224dbcbdca174d3e8648a8b5693fd130,VALUE=='crucial',1.5
3,HW,DEER,Mule Deer,Mule Deer Habitat,a4ea235cef824862b560c693b4f7959b,VALUE=='crucial',1.5
4,HW,MOOS,Moose,Moose Habitat,e66c061ebd4346e3b16ed8786f5d5d78,VALUE=='crucial',1.5
5,HW,BEAR,Black Bear,Black Bear Habitat,6440f19a05c149b783baf1705e5f2a51,VALUE=='crucial',1.5
6,HW,GOAT,Mountain Goat,Mountain Goat Habitat,b6b369f865d243399bfc80908426d31b,VALUE=='crucial',1.5
7,HW,DGRS,Dusky Grouse,Dusky Grouse Habitat,10382c7f5f384fc692d094fcb65bf251,VALUE=='crucial',1.5
8,HW,CHKR,Chukar,Chukar Habitat,97c6f7c74f704f84b687eb0746cc07ec,VALUE=='crucial',1.5
9,HW,BISN,Bison,Bison Habitat,9bed043eb87641db88424d1080d03045,VALUE=='crucial',1.5


In [7]:
#Check for duplicate codes:
dfOpenDataLayers['LayerCode'].duplicated().any()

False

In [8]:
#item1  = gis.content.get("a4ea235cef824862b560c693b4f7959b")
#item2  = gis.content.get("dbc6a99d04944dab9f23bb6286843a1d")
#flayer1 = item1.layers[0]
#flayer2 = item2.layers[0]
#sdfTemp1 = pd.DataFrame.spatial.from_layer(flayer1)
#sdfTemp2 = pd.DataFrame.spatial.from_layer(flayer1)
#
#sdfTemp1.query("VALUE=='crucial'")


In [10]:
bProcess         = True # toggle for entire cell... Set to false if you do not want to download and do not want to rejoin to projects
bReDownloadAll   = True # Set to True if you want to redownload data... otherwise set to False, this will download data and store in intermediate step... makes joining faster
bReJoinAll       = True  # Set to True if you want to rejoin the projects to the open data layers

gis = GIS(expiration=9999) # set expiration higher so it doesn't time out with the wetlands

fcRtpSeg = os.path.join(processGdb,'_rtp_proj_lines_segs')
fcRtpPnt = os.path.join(processGdb,'_rtp_proj_points')

if bProcess: 

    import time

    start_time = time.time()

    for index, row in dfOpenDataLayers.iterrows():

        # display all the variables used
        print (row['LayerCode'] + ' -  ' + row['OpenDataKey'] + ' - ' + str(row['SearchBuffer']) + ' -  ' + row['QueryFilter'])

        #try:
        _fc_download = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_orig_fltr' )
        _fc_seg_join = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_seg_joined')
        _fc_pnt_join = os.path.join(intermediate_folder,'process.gdb\\' + row['LayerCode'] + '_pnt_joined')

        # check to see if replace all toggle is on OR if the feature class already exists in the gdb
        if bReDownloadAll or not arcpy.Exists(_fc_download):

            # delete feature class if it exists
            delIfExists(_fc_download)

            # get data from web
            print ("downloading...")
            _item  = gis.content.get(row['OpenDataKey'])
            _flayer = _item.layers[0]
            _sdfLyr = pd.DataFrame.spatial.from_layer(_flayer)

            # filter data based on query
            print ("filtering...")
            if bool(row['QueryFilter']):
                _sdfLyr = _sdfLyr.query(row['QueryFilter'])

            # if sdf is not empty
            if not _sdfLyr.empty:
                print ("exporting intermediate layer...")
                _sdfLyr.spatial.to_featureclass(_fc_download)

        if bReJoinAll or not arcpy.Exists(_fc_seg_join) or not arcpy.Exists(_fc_pnt_join):

            # only join again if download has been created
            if arcpy.Exists(_fc_download):

                # delete feature class if it exists
                delIfExists(_fc_seg_join)
                delIfExists(_fc_pnt_join)

                # get closest distance
                print ("joining...")
                arcpy.analysis.SpatialJoin(fcRtpSeg, _fc_download, _fc_seg_join, "JOIN_ONE_TO_ONE", "KEEP_COMMON", match_option="CLOSEST", search_radius=str(row['SearchBuffer']) + " mile", distance_field_name="meters" + row['LayerCode'])
                arcpy.analysis.SpatialJoin(fcRtpPnt, _fc_download, _fc_pnt_join, "JOIN_ONE_TO_ONE", "KEEP_COMMON", match_option="CLOSEST", search_radius=str(row['SearchBuffer']) + " mile", distance_field_name="meters" + row['LayerCode'])

            else:
                print('no features to join to')
                
        end_time = time.time()

        hours, rem = divmod(end_time - start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"done... elapsed time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds") 
        #except:
        #    print ("ERROR: " + row['LayerCode'])

print ('ALL DONE!')

LOWN -  929fe39efa1d43ac8b151c2898ad3efe - 1.5 -  OWNER!='Private'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 0 minutes, 49 seconds
AGFL -  875527c1e41e40d8b52340f18e89fb52 - 1.5 -  
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 23 seconds
ELKS -  224dbcbdca174d3e8648a8b5693fd130 - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 35 seconds
DEER -  a4ea235cef824862b560c693b4f7959b - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 48 seconds
MOOS -  e66c061ebd4346e3b16ed8786f5d5d78 - 1.5 -  VALUE=='crucial'
downloading...
filtering...
exporting intermediate layer...
joining...
done... elapsed time: 0 hours, 1 minutes, 56 seconds
BEAR -  6440f19a05c149b783baf1705e5f2a51 - 1.5 -  VALUE=='cru

In [ ]:
##import arcgis libraries
#from arcgis.gis import *
#gis = GIS()
#
##create map1 that centers on Salt Lake (can replace 'Salt Lake' with any place name or leave blank for entire world)
#map3 = gis.map('Salt Lake')
#
#sdf = pd.DataFrame.spatial.from_featureclass(os.path.join(intermediate_folder,'process.gdb\\WETL_joined'))
#
##plot sdf using class breaks and blue colors for 'columnname' field
##layer will be added to 'map1' above
#sdf.spatial.plot(map_widget = map3)
#
##show map1
#map3

In [11]:
dfD = pd.DataFrame()

for index, row in dfOpenDataLayers.iterrows():
    #try:
    print(row['LayerCode'])
    _fcSeg = os.path.join(processGdb,row['LayerCode'] + '_seg_joined')
    _fcPnt = os.path.join(processGdb,row['LayerCode'] + '_pnt_joined')
    if arcpy.Exists(_fcSeg) & arcpy.Exists(_fcPnt):
        #display(_fc)

        _sdfSeg = pd.DataFrame.spatial.from_featureclass(_fcSeg)
        _sdfSeg.columns = _sdfSeg.columns.str.upper()

        df = _sdfSeg[['GIS_ID',"METERS" + row['LayerCode'],'ORIG_SEQ']].copy()
        df['SEGORPNT' ] = 'seg'
        df['LAYERCODE'] = row['LayerCode']

        df.columns=(['GIS_ID','DIST_METERS','ORIG_SEQ','SEGORPNT','LAYERCODE'])

        #display(_sdf[['gis_id',"meters" + row['LayerCode']]])

        dfD = pd.concat([dfD,df])

        _sdfPnt = pd.DataFrame.spatial.from_featureclass(_fcPnt)
        _sdfPnt.columns = _sdfPnt.columns.str.upper()

        df = _sdfPnt[['GIS_ID',"METERS" + row['LayerCode']]].copy()
        df['ORIG_SEQ' ] = 0 # points have no sequence so set to 0
        df['SEGORPNT' ] = 'pnt'
        df['LAYERCODE'] = row['LayerCode']

        df.columns=(['GIS_ID','DIST_METERS','ORIG_SEQ','SEGORPNT','LAYERCODE'])

        #display(_sdf[['gis_id',"meters" + row['LayerCode']]])

        dfD = pd.concat([dfD,df])

dfD['DIST_FEET'] = dfD['DIST_METERS'] / 3.28084

dfD = dfD.drop(columns=('DIST_METERS'))

dfD

LOWN
AGFL
ELKS
DEER
MOOS
BEAR
GOAT
DGRS
CHKR
BISN
CSTG
CAQU
CABS
BTPG
BFWT
BFNT
HZWO
SWLN
FUDS
MMRP
NPRL
SFST
ICBZ
TXRI
VCUP
UHDS
EQFA
LCPK
PAPR
FLDP
QUFL
STSL
RIPA
WETL


,GIS_ID,ORIG_SEQ,SEGORPNT,LAYERCODE,DIST_FEET
0,A-B-1,1,seg,LOWN,451.673115
1,A-B-1,2,seg,LOWN,458.791283
2,A-B-1,7,seg,LOWN,689.511914
3,A-B-1,6,seg,LOWN,635.915899
4,A-B-1,5,seg,LOWN,583.716840
...,...,...,...,...,...
212,689,0,pnt,WETL,108.296556
213,706,0,pnt,WETL,186.416822
214,218,0,pnt,WETL,70.746989
215,721,0,pnt,WETL,8.773822


In [12]:
# DON'T USE NONE OR NEW
dfDistance = dfD[['SEGORPNT','GIS_ID','DIST_FEET','ORIG_SEQ','LAYERCODE']].copy()
dfDistance.columns =['x','g','ft','s','l']
dfDistance['id'] = dfDistance['g'].astype(str) + '_' + dfDistance['s'].astype(str)
dfDistance = dfDistance[['x','l','id','ft']]
dfDistance = dfDistance.groupby(['x','l','id'],as_index=False).agg(ft=('ft','mean'))
dfDistance['ft'] = dfDistance['ft'].astype(int)
dfDistance

,x,l,id,ft
0,pnt,AGFL,137_0,40
1,pnt,AGFL,138_0,26
2,pnt,AGFL,139_0,61
3,pnt,AGFL,140_0,577
4,pnt,AGFL,142_0,367
...,...,...,...,...
303661,seg,WETL,WestWeber_86,0
303662,seg,WETL,WestWeber_87,0
303663,seg,WETL,WestWeber_88,0
303664,seg,WETL,WestWeber_89,0


In [13]:
dfDistance_pivot = dfD.pivot_table(index=['SEGORPNT','GIS_ID','ORIG_SEQ'],columns='LAYERCODE',aggfunc='mean')
dfDistance_pivot.fillna(-1,inplace=True)
dfDistance_pivot = dfDistance_pivot.astype(int)
dfDistance_pivot.columns = dfDistance_pivot.columns.droplevel(0)
dfDistance_pivot.reset_index(inplace=True)
dfDistance_pivot

LAYERCODE,SEGORPNT,GIS_ID,ORIG_SEQ,AGFL,BEAR,BFNT,BFWT,BTPG,CAQU,CHKR,...,PAPR,QUFL,RIPA,SFST,STSL,SWLN,TXRI,UHDS,VCUP,WETL
0,pnt,137,0,40,-1,-1,-1,-1,99,-1,...,-1,-1,-1,368,376,357,-1,-1,-1,102
1,pnt,138,0,26,-1,-1,-1,-1,117,-1,...,-1,-1,-1,492,-1,684,-1,-1,-1,10
2,pnt,139,0,61,-1,-1,-1,-1,23,-1,...,235,-1,-1,279,-1,406,-1,-1,-1,79
3,pnt,140,0,577,-1,-1,703,-1,0,-1,...,-1,111,675,66,-1,586,196,-1,192,32
4,pnt,142,0,367,-1,-1,-1,-1,167,-1,...,-1,473,-1,269,-1,-1,353,-1,-1,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25564,seg,WestWeber,85,2,-1,-1,-1,-1,-1,-1,...,452,-1,-1,-1,-1,-1,-1,-1,-1,7
25565,seg,WestWeber,86,3,-1,-1,-1,-1,-1,-1,...,450,-1,-1,-1,-1,-1,-1,-1,-1,0
25566,seg,WestWeber,87,59,-1,-1,-1,-1,-1,-1,...,460,-1,-1,-1,-1,-1,-1,-1,-1,0
25567,seg,WestWeber,88,98,-1,-1,-1,-1,-1,-1,...,495,-1,-1,722,-1,-1,-1,-1,-1,0


In [14]:
dfDistance_pivot[dfDistance_pivot['GIS_ID']=='83']

LAYERCODE,SEGORPNT,GIS_ID,ORIG_SEQ,AGFL,BEAR,BFNT,BFWT,BTPG,CAQU,CHKR,...,PAPR,QUFL,RIPA,SFST,STSL,SWLN,TXRI,UHDS,VCUP,WETL


In [15]:
dfDistance_pivot.max()

LAYERCODE
SEGORPNT          seg
GIS_ID      WestWeber
ORIG_SEQ          424
AGFL              735
BEAR              734
BFNT              735
BFWT              735
BTPG              735
CAQU              735
CHKR              735
DEER              735
DGRS              735
ELKS              735
EQFA              735
FLDP              735
FUDS              735
GOAT              735
HZWO              735
ICBZ              735
LCPK              735
LOWN              735
MMRP              734
MOOS              735
NPRL              735
PAPR              735
QUFL              735
RIPA              735
SFST              735
STSL              735
SWLN              735
TXRI              735
UHDS              735
VCUP              735
WETL              735
dtype: object

# Web App Data

In [16]:
sdfRtpSeg = pd.DataFrame.spatial.from_featureclass(fcRtpSeg)
sdfRtpSeg.columns = sdfRtpSeg.columns.str.upper()
sdfRtpPnt = pd.DataFrame.spatial.from_featureclass(fcRtpPnt)
sdfRtpPnt.columns = sdfRtpPnt.columns.str.upper()

sdfRtpSeg = sdfRtpSeg.groupby(['GIS_ID','MODE','ORIG_SEQ'],as_index=False).agg(SHAPE=('SHAPE','first'))
sdfRtpPnt = sdfRtpPnt.groupby(['GIS_ID','MODE'           ],as_index=False).agg(SHAPE=('SHAPE','first'))

sdfRtpSegWithDist = pd.DataFrame.merge(sdfRtpSeg,dfDistance_pivot[dfDistance_pivot['SEGORPNT']=='seg'],on=('GIS_ID','ORIG_SEQ'),how='left')
sdfRtpPntWithDist = pd.DataFrame.merge(sdfRtpPnt,dfDistance_pivot[dfDistance_pivot['SEGORPNT']=='pnt'],on=('GIS_ID'           ),how='left')

# iterate through the list of columns and add them to the DataFrame
for col in dfOpenDataLayers['LayerCode'].unique():
    if col not in sdfRtpSegWithDist.columns:
        sdfRtpSegWithDist[col] = None
    if col not in sdfRtpPntWithDist.columns:
        sdfRtpPntWithDist[col] = None
        
sdfRtpSegWithDist.fillna(-1,inplace=True)
sdfRtpPntWithDist.fillna(-1,inplace=True)

display(sdfRtpSegWithDist)
display(sdfRtpPntWithDist)

,GIS_ID,MODE,ORIG_SEQ,SHAPE,SEGORPNT,AGFL,BEAR,BFNT,BFWT,BTPG,...,SFST,STSL,SWLN,TXRI,UHDS,VCUP,WETL,BISN,CSTG,CABS
0,10,Highway,1,"{'paths': [[[413426.50009999983, 4507190.0601]...",seg,133,-1,311,-1,-1,...,304,-1,-1,-1,-1,-1,56,-1,-1,-1
1,10,Highway,2,"{'paths': [[[413225.3596999999, 4507186.7063],...",seg,81,-1,311,-1,-1,...,302,-1,-1,-1,-1,-1,50,-1,-1,-1
2,10,Highway,3,"{'paths': [[[413024.19849999994, 4507185.1808]...",seg,52,-1,317,-1,-1,...,303,-1,-1,-1,-1,-1,59,-1,-1,-1
3,10,Highway,4,"{'paths': [[[412823.03369999956, 4507183.9804]...",seg,50,-1,335,-1,-1,...,315,-1,-1,-1,-1,-1,102,-1,-1,-1
4,10,Highway,5,"{'paths': [[[412626.7885999996, 4507150.073899...",seg,60,-1,371,-1,-1,...,282,-1,-1,-1,-1,-1,116,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25347,WestWeber,Transit,85,"{'paths': [[[401672.7982000001, 4566569.049000...",seg,2,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,7,-1,-1,-1
25348,WestWeber,Transit,86,"{'paths': [[[401471.69130000006, 4566573.3893]...",seg,3,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,0,-1,-1,-1
25349,WestWeber,Transit,87,"{'paths': [[[401285.83779999986, 4566643.4121]...",seg,59,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,0,-1,-1,-1
25350,WestWeber,Transit,88,"{'paths': [[[401164.80399999954, 4566800.8816]...",seg,98,-1,-1,-1,-1,...,722,-1,-1,-1,-1,-1,0,-1,-1,-1


,GIS_ID,MODE,SHAPE,SEGORPNT,ORIG_SEQ,AGFL,BEAR,BFNT,BFWT,BTPG,...,SFST,STSL,SWLN,TXRI,UHDS,VCUP,WETL,BISN,CSTG,CABS
0,137,Highway,"{'x': 406246.0999999996, 'y': 4508061.2895, 's...",pnt,0,40,-1,-1,-1,-1,...,368,376,357,-1,-1,-1,102,-1,-1,-1
1,138,Highway,"{'x': 407788.95660000015, 'y': 4507995.2905, '...",pnt,0,26,-1,-1,-1,-1,...,492,-1,684,-1,-1,-1,10,-1,-1,-1
2,139,Highway,"{'x': 410203.63999999966, 'y': 4508060.15, 'sp...",pnt,0,61,-1,-1,-1,-1,...,279,-1,406,-1,-1,-1,79,-1,-1,-1
3,140,Highway,"{'x': 419670.39609999955, 'y': 4508695.5358000...",pnt,0,577,-1,-1,703,-1,...,66,-1,586,196,-1,192,32,-1,-1,-1
4,142,Highway,"{'x': 413493.3624, 'y': 4513848.4355, 'spatial...",pnt,0,367,-1,-1,-1,-1,...,269,-1,-1,353,-1,-1,105,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,T-S-78,Transit,"{'x': 429062.5, 'y': 4502006.699999999, 'spati...",pnt,0,144,-1,-1,-1,-1,...,3,-1,-1,-1,-1,-1,189,-1,-1,-1
213,T-S-79,Transit,"{'x': 429471.23060000036, 'y': 4492452.8686999...",pnt,0,550,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,638,-1,-1,-1
214,T-S-80,Transit,"{'x': 414132.61000000034, 'y': 4488638.09, 'sp...",pnt,0,80,-1,-1,-1,-1,...,729,-1,-1,-1,-1,-1,105,-1,-1,-1
215,T-W-13,Transit,"{'x': 429484.3501000004, 'y': 4567268.1401, 's...",pnt,0,687,403,-1,-1,-1,...,-1,10,-1,-1,-1,-1,19,-1,-1,-1


In [17]:
# export segs
bExportSegs = True

if bExportSegs:
    sdfRtpSegWithDist.spatial.to_featureclass(os.path.join(results_folder,"resiliency_seg.shp"), sanitize_columns=False)
    sdfRtpPntWithDist.spatial.to_featureclass(os.path.join(results_folder,"resiliency_pnt.shp"), sanitize_columns=False)
    print("SEG/PNT EXPORTED")
    sdfRtpLines  = pd.DataFrame.spatial.from_featureclass(os.path.join(processGdb, '_rtp_proj_lines' ))
    sdfRtpPoints = pd.DataFrame.spatial.from_featureclass(os.path.join(processGdb, '_rtp_proj_points'))
    sdfRtpLines .columns = sdfRtpLines .columns.str.upper()
    sdfRtpPoints.columns = sdfRtpPoints.columns.str.upper()
    sdfRtpLines  = sdfRtpLines [['GIS_ID','SHAPE']]
    sdfRtpPoints = sdfRtpPoints[['GIS_ID','SHAPE']]
    sdfRtpLines .spatial.to_featureclass(os.path.join(results_folder,"resiliency_projects_lines.shp" ), sanitize_columns=False)
    sdfRtpPoints.spatial.to_featureclass(os.path.join(results_folder,"resiliency_projects_points.shp"), sanitize_columns=False)
    print("LINES/POINTS EXPORTED")
else:
    print("SEG/PNT NOT EXPORTED")
    print("LINES/POINTS NOT EXPORTED")


SEG/PNT EXPORTED
LINES/POINTS EXPORTED


In [19]:
dfDistance_pivot[['GIS_ID']].drop_duplicates()

LAYERCODE,GIS_ID
0,137
1,138
2,139
3,140
4,142
...,...
25163,StGrid
25244,StGrid_2
25350,StGrid_2_BRT
25467,TRAXImp


In [20]:
# export jsons for categories and layers
dfCategories    .to_json(os.path.join(folderWebAppDataFolder,'cats.json'),orient="table")
dfOpenDataLayers.to_json(os.path.join(folderWebAppDataFolder,'lyrs.json'),orient="table")

# get only gisid and seq of
pd.concat([sdfRtpSegWithDist[['GIS_ID','ORIG_SEQ','MODE']],sdfRtpPntWithDist[['GIS_ID','ORIG_SEQ','MODE']]]).drop_duplicates().rename(columns={'GIS_ID':'g','ORIG_SEQ':'s','MODE':'m'}).to_json(os.path.join(folderWebAppDataFolder,'segs.json'),index=False,orient="table")

# read original project data
dfPoints = pd.DataFrame.spatial.from_featureclass(os.path.join(processGdb, '_rtp_proj_points'))
dfLines  = pd.DataFrame.spatial.from_featureclass(os.path.join(processGdb, '_rtp_proj_lines' ))
dfPoints['SEGORPNT'] = 'pnt'
dfLines ['SEGORPNT'] = 'seg'
dfPoints.columns = dfPoints.columns.str.upper()
dfLines.columns  = dfLines.columns.str.upper()

# get all gisids
pd.concat([dfPoints[['GIS_ID','MODE','PLAN_ID','NAME','SEGORPNT']],dfLines[['GIS_ID','MODE','PLAN_ID','NAME','SEGORPNT']]]).drop_duplicates().rename(columns={'GIS_ID':'g','MODE':'m','PLAN_ID':'p','NAME':'n','SEGORPNT':'x'}).to_json(os.path.join(folderWebAppDataFolder,'gids.json'),index=False,orient="table")

In [21]:
sdfRtpSeg

,GIS_ID,MODE,ORIG_SEQ,SHAPE
0,10,Highway,1,"{""paths"": [[[413426.50009999983, 4507190.0601]..."
1,10,Highway,2,"{""paths"": [[[413225.3596999999, 4507186.7063],..."
2,10,Highway,3,"{""paths"": [[[413024.19849999994, 4507185.1808]..."
3,10,Highway,4,"{""paths"": [[[412823.03369999956, 4507183.9804]..."
4,10,Highway,5,"{""paths"": [[[412626.7885999996, 4507150.073899..."
...,...,...,...,...
25347,WestWeber,Transit,85,"{""paths"": [[[401672.7982000001, 4566569.049000..."
25348,WestWeber,Transit,86,"{""paths"": [[[401471.69130000006, 4566573.3893]..."
25349,WestWeber,Transit,87,"{""paths"": [[[401285.83779999986, 4566643.4121]..."
25350,WestWeber,Transit,88,"{""paths"": [[[401164.80399999954, 4566800.8816]..."


In [22]:
sdfRtpPnt[sdfRtpPnt['GIS_ID']=='83']

,GIS_ID,MODE,SHAPE


In [23]:
_df = pd.concat([sdfRtpSeg[['GIS_ID','MODE']],sdfRtpPnt[['GIS_ID','MODE']]])
_df[_df['GIS_ID']=='83']

,GIS_ID,MODE


In [24]:
# REDO

# FILE BY BUFFER - GIS_ID - CATEGORY
# ROW BY ORIG_SEQ
# RECORDS: 

lstBuffers = [100,200,300,400,500,600]

dfBuffers = dfDistance.copy()

for b in lstBuffers:
    print(b)
    dfBuffers.loc[(dfBuffers['ft']<b),'b' + str(b)] = 1
    dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].fillna(0)
    dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].astype(int)


display(dfBuffers)
dfBuffers.drop(columns=('ft'),inplace=True)

display(dfBuffers)
# export one dist file per gis id

#for index, row in dfDistance[['l']].drop_duplicates().iterrows():
#    print(row['l'])
#    _dfExport = dfDistance[dfDistance['l']==row['l']][['id','d']].set_index('id').to_json(os.path.join(folderWebAppDataFolder + '\\dist' , row['l'] + '.json'),orient='index')

100
200
300
400
500
600


,x,l,id,ft,b100,b200,b300,b400,b500,b600
0,pnt,AGFL,137_0,40,1,1,1,1,1,1
1,pnt,AGFL,138_0,26,1,1,1,1,1,1
2,pnt,AGFL,139_0,61,1,1,1,1,1,1
3,pnt,AGFL,140_0,577,0,0,0,0,0,1
4,pnt,AGFL,142_0,367,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
303661,seg,WETL,WestWeber_86,0,1,1,1,1,1,1
303662,seg,WETL,WestWeber_87,0,1,1,1,1,1,1
303663,seg,WETL,WestWeber_88,0,1,1,1,1,1,1
303664,seg,WETL,WestWeber_89,0,1,1,1,1,1,1


,x,l,id,b100,b200,b300,b400,b500,b600
0,pnt,AGFL,137_0,1,1,1,1,1,1
1,pnt,AGFL,138_0,1,1,1,1,1,1
2,pnt,AGFL,139_0,1,1,1,1,1,1
3,pnt,AGFL,140_0,0,0,0,0,0,1
4,pnt,AGFL,142_0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...
303661,seg,WETL,WestWeber_86,1,1,1,1,1,1
303662,seg,WETL,WestWeber_87,1,1,1,1,1,1
303663,seg,WETL,WestWeber_88,1,1,1,1,1,1
303664,seg,WETL,WestWeber_89,1,1,1,1,1,1


In [25]:
dfBuffersWithCat = pd.DataFrame.merge(dfBuffers, dfOpenDataLayers, left_on='l', right_on='LayerCode')
#display(dfBuffersWithCat)

for b in lstBuffers:
    print(b)
    for c in dfCategories['CategoryCode'].unique():
        print(c)
        _lyrs = (dfOpenDataLayers[dfOpenDataLayers['CategoryCode']==c]['LayerCode'].unique())
        _df = dfBuffersWithCat[(dfBuffersWithCat['CategoryCode']==c)][['id','l','CategoryCode','b' + str(b)]]
        _df = _df[_df['b' + str(b)]>0]
        _df.loc[(_df['b' + str(b)]==1),'value'] = _df['l']
        #display(_df)

        _dfp = _df.pivot(index='id', columns='l', values='value')
        _dfp.reset_index(inplace=True)
        _dfp.fillna('',inplace=True)
        _dfp[c] = _dfp[_df['l'].unique()].apply(lambda x: ",".join(filter(None, x)), axis =1)
        _dfp = _dfp[['id',c]]
        _dfp.set_index('id',inplace=True)
        #display(_dfp)

        _dfp.to_json(os.path.join(folderWebAppDataFolder + '\\within_buffers' , 'b' + str(b) + '_' + c + '.json'),orient='index')

100
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
200
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
300
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
400
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
500
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
600
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
